# Test Load Daily

Start by loading one "new" tile.

## Imports

In [ ]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
from sqlalchemy.exc import IntegrityError
from desiutil.log import get_logger, DEBUG
from desispec.io.meta import findfile
import specprodDB.load as db
import specprodDB.tile as t

## Initial Values

In [ ]:
specprod = os.environ['SPECPROD']
# tile_id, tile_survey, tile_program = 3867, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5053, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5052, 'main', 'dark'
# tile_id, tile_survey, tile_program = 5074, 'main', 'dark'
# tile_id, tile_survey, tile_program = 1685, 'main', 'dark'
# tile_id, tile_survey, tile_program = 40069, 'main', 'backup'
# tile_id, tile_survey, tile_program = 80950, 'sv1', 'backup'
overwrite = True

## Initialize Database

In [ ]:
db.log = get_logger(DEBUG)
postgresql = db.setup_db(schema=specprod, hostname='db-loadbalancer.bweaver.development.svc.spin.nersc.org', username='desi_admin', overwrite=overwrite)
if overwrite:
    db.load_versions('computed', 'daily/v0', 'daily', specprod, 'main')

## Read tiles file

In [ ]:
# tiles_file = findfile('tiles', readonly=True).replace('.fits', '.csv')
tiles_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv')
tiles_table = Table.read(tiles_file, format='ascii.csv')
# tiles_table

In [ ]:
row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0) & (~tiles_table['PROGRAM'].mask))[0]
tiles_table[row_index]
# row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0))[0]
# tiles_table[row_index][tiles_table[row_index]['PROGRAM'].mask]

In [ ]:
tiles_table[row_index][:20]

In [ ]:
load_tiles = db.Tile.convert(tiles_table, row_index=row_index)

In [ ]:
load_tiles[:20]

In [ ]:
try:
    db.dbSession.add_all(load_tiles)
    db.dbSession.commit()
except IntegrityError:
    print("ERROR: Some tiles are already loaded!")
    db.dbSession.rollback()

## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these.

In [ ]:
# exposures_file = findfile('exposures', readonly=True)
exposures_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits')
exposures_table = Table.read(exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(exposures_file, format='fits', hdu='FRAMES')
# exposures_table[exposures_table['TILEID'] == new_tile.tileid]

In [ ]:
load_exposures = list()
for new_tile in load_tiles:
    row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        # load_exposures += db.Exposure.convert(exposures_table, row_index=row_index)
        if (exposures_table[row_index]['MJD'] < 50000).any():
            print("WARNING: Invalid MJD values detected for tile {0:d}!".format(new_tile.tileid))
            bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0) & (exposures_table['MJD'] < 50000))[0]
            for row in exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index]:
                raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
                with fits.open(raw_data_file, mode='readonly') as hdulist:
                    mjd_obs = hdulist['SPEC'].header['MJD-OBS']
                    print("WARNING: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(new_tile.tileid, row['EXPID'], mjd_obs, raw_data_file))
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid))[0]
        print(exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
# load_frames = list()
# for exposure in load_exposures:
#     row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
#     assert len(row_index) > 0
#     load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_exposures

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_exposures)
db.dbSession.commit()

In [ ]:
# db.dbSession.rollback()
db.dbSession.add_all(load_frames)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [ ]:
potential_targets_table = t.potential_targets(load_tiles[1].tileid)
# potential_targets_table

In [ ]:
potential_cat = t.potential_photometry(load_tiles[1], potential_targets_table)
# potential_cat

In [ ]:
potential_targetphot = t.targetphot(potential_cat)
potential_tractorphot = t.tractorphot(potential_cat)

In [ ]:
load_photometry = t.load_photometry(potential_tractorphot)

### Load photometry, such as it is, for objects that are not in the tractor catalog

In [ ]:
load_targetphot = t.load_targetphot(potential_targetphot, load_photometry)

### Load the target table

In [ ]:
load_target = t.load_target(load_tiles[1], potential_targetphot)

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [ ]:
load_ztile = t.load_redshift(load_tiles[0])

## Load fiberassign and potential

In [ ]:
load_fiberassign, load_potential = t.load_fiberassign(load_tiles[0])

## Recompute global values

The global values are the primary classification and number of spectra.

In [ ]:
t.update_primary()

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [ ]:
t.update_q3c()